<a href="https://colab.research.google.com/github/draginverse/dragin-healthcare/blob/feature%2Fg-retriever/scripts/retrieval/gretriever.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install torch_geometric
!pip install pcst_fast
!pip install torch_scatter -f https://data.pyg.org/
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.3/243.3 kB 17.3 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for torch_scatter: filename=torch_scatter-2.1.2-cp311-cp311-linux_x86_64.whl size=547368 sha256=803e1674e02c75536da80267d88b4ae60dde2ca1def1eec73ac962402fabc28e
  Stored in directory: /root/.cache/pip/wheels/b8/d4/0e/a80af2465354ea7355a2c153b11af2da739cfcf08b6c0b28e2
Successfully built torch_scatter
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19

# simulate graphs input

In [5]:
toy_graphs = [
    {
        "nodes": ["Lung", "Alveoli", "Oxygen", "Carbon dioxide", "Bloodstream"],
        "edges": [
            ("Lung", "contains", "Alveoli"),
            ("Alveoli", "exchange", "Oxygen"),
            ("Alveoli", "remove", "Carbon dioxide"),
            ("Oxygen", "enters", "Bloodstream")
        ]
    },
    {
        "nodes": ["Asthma", "Airways", "Inflammation", "Bronchodilator"],
        "edges": [
            ("Asthma", "affects", "Airways"),
            ("Airways", "show", "Inflammation"),
            ("Bronchodilator", "relieves", "Inflammation")
        ]
    },
    {
        "nodes": ["Pneumonia", "Lung infection", "Bacteria", "Antibiotics"],
        "edges": [
            ("Pneumonia", "is a", "Lung infection"),
            ("Lung infection", "caused by", "Bacteria"),
            ("Antibiotics", "treat", "Bacteria")
        ]
    },
    {
        "nodes": ["Chronic obstructive pulmonary disease (COPD)", "Smoking", "Shortness of breath", "Lung damage"],
        "edges": [
            ("Smoking", "leads to", "Chronic obstructive pulmonary disease (COPD)"),
            ("Chronic obstructive pulmonary disease (COPD)", "causes", "Shortness of breath"),
            ("Chronic obstructive pulmonary disease (COPD)", "results in", "Lung damage")
        ]
    },
    {
        "nodes": ["Bronchitis", "Cough", "Mucus", "Viral infection"],
        "edges": [
            ("Viral infection", "can cause", "Bronchitis"),
            ("Bronchitis", "leads to", "Cough"),
            ("Cough", "produces", "Mucus")
        ]
    },
    {
        "nodes": ["Pulmonary embolism", "Blood clot", "Lung arteries", "Chest pain"],
        "edges": [
            ("Blood clot", "blocks", "Lung arteries"),
            ("Blood clot", "leads to", "Pulmonary embolism"),
            ("Pulmonary embolism", "causes", "Chest pain")
        ]
    }
]

'''     THIS IS FOR TRYING TO BUILD THE GRAPHS
import pandas as pd
import torch
from datasets import load_dataset, concatenate_datasets
from torch_geometric.data import Data
from src.utils.lm_modeling import load_model, load_text2embedding

def create_graphs_in_memory():
    dataset = load_dataset("rmanluo/RoG-webqsp")
    dataset = concatenate_datasets([dataset['train'], dataset['validation'], dataset['test']])

    model_name = 'sbert'
    model, tokenizer, device = load_model[model_name]()
    text2embedding = load_text2embedding[model_name]

    graphs = []         # list of Data objects (PyG graphs)
    textual_nodes = []  # list of pandas DataFrames (for node attributes)
    textual_edges = []  # list of pandas DataFrames (for edge attributes)

    for i in range(len(dataset)):
        nodes = {}
        edges = []
        for h, r, t in dataset[i]['graph']:
            h, t = h.lower(), t.lower()
            if h not in nodes:
                nodes[h] = len(nodes)
            if t not in nodes:
                nodes[t] = len(nodes)
            edges.append({'src': nodes[h], 'edge_attr': r, 'dst': nodes[t]})

        nodes_df = pd.DataFrame(
            [{'node_id': v, 'node_attr': k} for k, v in nodes.items()],
            columns=['node_id', 'node_attr']
        )
        edges_df = pd.DataFrame(edges, columns=['src', 'edge_attr', 'dst'])

        if len(nodes_df) == 0:
            continue  # skip empty graphs

        # embeddings
        x = text2embedding(model, tokenizer, device, nodes_df.node_attr.tolist())
        edge_attr = text2embedding(model, tokenizer, device, edges_df.edge_attr.tolist())
        edge_index = torch.LongTensor([edges_df.src.tolist(), edges_df.dst.tolist()])

        pyg_graph = Data(x=x, edge_index=edge_index, edge_attr=edge_attr, num_nodes=len(nodes_df))

        graphs.append(pyg_graph)
        textual_nodes.append(nodes_df)
        textual_edges.append(edges_df)

    return graphs, textual_nodes, textual_edges
'''

'     THIS IS FOR TRYING TO BUILD THE GRAPHS\nimport pandas as pd\nimport torch\nfrom datasets import load_dataset, concatenate_datasets\nfrom torch_geometric.data import Data\nfrom src.utils.lm_modeling import load_model, load_text2embedding\n\ndef create_graphs_in_memory():\n    dataset = load_dataset("rmanluo/RoG-webqsp")\n    dataset = concatenate_datasets([dataset[\'train\'], dataset[\'validation\'], dataset[\'test\']])\n\n    model_name = \'sbert\'\n    model, tokenizer, device = load_model[model_name]()\n    text2embedding = load_text2embedding[model_name]\n\n    graphs = []         # list of Data objects (PyG graphs)\n    textual_nodes = []  # list of pandas DataFrames (for node attributes)\n    textual_edges = []  # list of pandas DataFrames (for edge attributes)\n\n    for i in range(len(dataset)):\n        nodes = {}\n        edges = []\n        for h, r, t in dataset[i][\'graph\']:\n            h, t = h.lower(), t.lower()\n            if h not in nodes:\n                nod

In [27]:
import torch
from torch_geometric.data import Data, Batch
from sklearn.preprocessing import LabelEncoder

def transform_graphs_to_pyg(toy_graphs, feature_dim=384):
    # Collect all nodes and edge types
    all_nodes = list({node for graph in toy_graphs for node in graph["nodes"]})
    all_edge_types = list({edge[1] for graph in toy_graphs for edge in graph["edges"]})

    # Create encoders
    node_encoder = {name: idx for idx, name in enumerate(all_nodes)}
    edge_type_encoder = {name: idx for idx, name in enumerate(all_edge_types)}

    pyg_graphs = []
    for graph in toy_graphs:
        # Create node features (!!! random for now !!!)
        num_nodes = len(graph["nodes"])
        x = torch.randn(num_nodes, feature_dim)

        # Process edges
        edge_list = []
        edge_attr_list = []
        for src, edge_type, dst in graph["edges"]:
            src_idx = graph["nodes"].index(src)  # Local graph index
            dst_idx = graph["nodes"].index(dst)  # Local graph index
            edge_list.append([src_idx, dst_idx])
            edge_attr_list.append(edge_type_encoder[edge_type])

        edge_index = torch.tensor(edge_list, dtype=torch.long).t().contiguous()
        edge_attr = torch.tensor(edge_attr_list, dtype=torch.long)

        pyg_graphs.append(Data(x=x, edge_index=edge_index, edge_attr=edge_attr))

    return pyg_graphs, node_encoder, edge_type_encoder

# Transform the graphs
pyg_graphs, node_encoder, edge_type_encoder = transform_graphs_to_pyg(toy_graphs)

# Create a batch of graphs for processing
toy_graph_batch = Batch.from_data_list(pyg_graphs)

# Print information about the first graph
print("First graph in PyG format:")
print(pyg_graphs[0])
#print("\nNode mapping:", {name: idx for idx, name in enumerate(node_encoder.classes_)})
#print("Edge type mapping:", {name: idx for idx, name in enumerate(edge_type_encoder.classes_)})
print("\nBatch information:")
print(toy_graph_batch)
print("Batch vector:", toy_graph_batch.batch)

print("Total nodes:", toy_graph_batch.num_nodes)
print("Batch vector max index:", toy_graph_batch.batch.max())
print("Batch vector length:", len(toy_graph_batch.batch))

# Verify consistency
assert toy_graph_batch.batch.max() < len(pyg_graphs), "Batch indices exceed graph count"
assert len(toy_graph_batch.batch) == toy_graph_batch.num_nodes, "Batch vector length mismatch"

First graph in PyG format:
Data(x=[5, 384], edge_index=[2, 4], edge_attr=[4])

Batch information:
DataBatch(x=[25, 384], edge_index=[2, 19], edge_attr=[19], batch=[25], ptr=[7])
Batch vector: tensor([0, 0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5,
        5])
Total nodes: 25
Batch vector max index: tensor(5)
Batch vector length: 25


# gnn.py

In [6]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, TransformerConv, GATConv


class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers, dropout, num_heads=-1):
        super(GCN, self).__init__()
        self.convs = torch.nn.ModuleList()
        self.convs.append(GCNConv(in_channels, hidden_channels))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(GCNConv(hidden_channels, hidden_channels))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.convs.append(GCNConv(hidden_channels, out_channels))
        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x, adj_t, edge_attr):
        for i, conv in enumerate(self.convs[:-1]):
            x = conv(x, adj_t)
            x = self.bns[i](x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, adj_t)
        return x, edge_attr


class GraphTransformer(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers, dropout, num_heads=-1):
        super(GraphTransformer, self).__init__()
        self.convs = torch.nn.ModuleList()
        self.convs.append(TransformerConv(in_channels=in_channels, out_channels=hidden_channels//num_heads, heads=num_heads, edge_dim=in_channels, dropout=dropout))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(TransformerConv(in_channels=hidden_channels, out_channels=hidden_channels//num_heads, heads=num_heads, edge_dim=in_channels, dropout=dropout,))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.convs.append(TransformerConv(in_channels=hidden_channels, out_channels=out_channels//num_heads, heads=num_heads, edge_dim=in_channels, dropout=dropout,))
        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x, adj_t, edge_attr):
        for i, conv in enumerate(self.convs[:-1]):
            x = conv(x, edge_index=adj_t, edge_attr=edge_attr)
            x = self.bns[i](x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, edge_index=adj_t, edge_attr=edge_attr)
        return x, edge_attr

class GAT(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers, dropout, num_heads=4):
        super(GAT, self).__init__()
        self.convs = torch.nn.ModuleList()
        self.convs.append(GATConv(in_channels, hidden_channels, heads=num_heads, concat=False))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(GATConv(hidden_channels, hidden_channels, heads=num_heads, concat=False))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.convs.append(GATConv(hidden_channels, out_channels, heads=num_heads, concat=False))
        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x, edge_index, edge_attr):
        for i, conv in enumerate(self.convs[:-1]):
            x = conv(x, edge_index=edge_index, edge_attr=edge_attr)
            x = self.bns[i](x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x,edge_index=edge_index, edge_attr=edge_attr)
        return x, edge_attr


load_gnn_model = {
    'gcn': GCN,
    'gat': GAT,
    'gt': GraphTransformer,
}

# graph_encoder.py

In [7]:
import torch
import torch.nn as nn
from torch_scatter import scatter
#from gnn import load_gnn_model

class GraphEncoder(nn.Module):
    def __init__(self, args):
        super().__init__()
        self.graph_encoder = load_gnn_model[args.gnn_model_name](
            in_channels=args.gnn_in_dim,
            out_channels=args.gnn_hidden_dim,
            hidden_channels=args.gnn_hidden_dim,
            num_layers=args.gnn_num_layers,
            dropout=args.gnn_dropout,
            num_heads=args.gnn_num_heads,
        )
        self.projector = nn.Sequential(
            nn.Linear(args.gnn_hidden_dim, 2048),
            nn.Sigmoid(),
            nn.Linear(2048, 4096),
        )

    def encode(self, graphs):
        graphs = graphs.to(next(self.parameters()).device)
        n_embeds, _ = self.graph_encoder(graphs.x, graphs.edge_index.long(), graphs.edge_attr)
        g_embeds = scatter(n_embeds, graphs.batch, dim=0, reduce='mean')
        projected_embeds = self.projector(g_embeds)
        return projected_embeds

# pcst_retrieval.py

In [8]:
import torch
import numpy as np
from pcst_fast import pcst_fast
from torch_geometric.data.data import Data


def retrieval_via_pcst(graph, q_emb, textual_nodes, textual_edges, topk=3, topk_e=3, cost_e=0.5):
    c = 0.01
    if len(textual_nodes) == 0 or len(textual_edges) == 0:
        desc = textual_nodes.to_csv(index=False) + '\n' + textual_edges.to_csv(index=False, columns=['src', 'edge_attr', 'dst'])
        graph = Data(x=graph.x, edge_index=graph.edge_index, edge_attr=graph.edge_attr, num_nodes=graph.num_nodes)
        return graph, desc

    root = -1  # unrooted
    num_clusters = 1
    pruning = 'gw'
    verbosity_level = 0
    if topk > 0:
        n_prizes = torch.nn.CosineSimilarity(dim=-1)(q_emb, graph.x)
        topk = min(topk, graph.num_nodes)
        _, topk_n_indices = torch.topk(n_prizes, topk, largest=True)

        n_prizes = torch.zeros_like(n_prizes)
        n_prizes[topk_n_indices] = torch.arange(topk, 0, -1).float()
    else:
        n_prizes = torch.zeros(graph.num_nodes)

    if topk_e > 0:
        e_prizes = torch.nn.CosineSimilarity(dim=-1)(q_emb, graph.edge_attr)
        topk_e = min(topk_e, e_prizes.unique().size(0))

        topk_e_values, _ = torch.topk(e_prizes.unique(), topk_e, largest=True)
        e_prizes[e_prizes < topk_e_values[-1]] = 0.0
        last_topk_e_value = topk_e
        for k in range(topk_e):
            indices = e_prizes == topk_e_values[k]
            value = min((topk_e-k)/sum(indices), last_topk_e_value)
            e_prizes[indices] = value
            last_topk_e_value = value*(1-c)
        # reduce the cost of the edges such that at least one edge is selected
        cost_e = min(cost_e, e_prizes.max().item()*(1-c/2))
    else:
        e_prizes = torch.zeros(graph.num_edges)

    costs = []
    edges = []
    vritual_n_prizes = []
    virtual_edges = []
    virtual_costs = []
    mapping_n = {}
    mapping_e = {}
    for i, (src, dst) in enumerate(graph.edge_index.T.numpy()):
        prize_e = e_prizes[i]
        if prize_e <= cost_e:
            mapping_e[len(edges)] = i
            edges.append((src, dst))
            costs.append(cost_e - prize_e)
        else:
            virtual_node_id = graph.num_nodes + len(vritual_n_prizes)
            mapping_n[virtual_node_id] = i
            virtual_edges.append((src, virtual_node_id))
            virtual_edges.append((virtual_node_id, dst))
            virtual_costs.append(0)
            virtual_costs.append(0)
            vritual_n_prizes.append(prize_e - cost_e)

    prizes = np.concatenate([n_prizes, np.array(vritual_n_prizes)])
    num_edges = len(edges)
    if len(virtual_costs) > 0:
        costs = np.array(costs+virtual_costs)
        edges = np.array(edges+virtual_edges)

    vertices, edges = pcst_fast(edges, prizes, costs, root, num_clusters, pruning, verbosity_level)

    selected_nodes = vertices[vertices < graph.num_nodes]
    selected_edges = [mapping_e[e] for e in edges if e < num_edges]
    virtual_vertices = vertices[vertices >= graph.num_nodes]
    if len(virtual_vertices) > 0:
        virtual_vertices = vertices[vertices >= graph.num_nodes]
        virtual_edges = [mapping_n[i] for i in virtual_vertices]
        selected_edges = np.array(selected_edges+virtual_edges)

    edge_index = graph.edge_index[:, selected_edges]
    selected_nodes = np.unique(np.concatenate([selected_nodes, edge_index[0].numpy(), edge_index[1].numpy()]))

    n = textual_nodes.iloc[selected_nodes]
    e = textual_edges.iloc[selected_edges]
    desc = n.to_csv(index=False)+'\n'+e.to_csv(index=False, columns=['src', 'edge_attr', 'dst'])

    mapping = {n: i for i, n in enumerate(selected_nodes.tolist())}

    x = graph.x[selected_nodes]
    edge_attr = graph.edge_attr[selected_edges]
    src = [mapping[i] for i in edge_index[0].tolist()]
    dst = [mapping[i] for i in edge_index[1].tolist()]
    edge_index = torch.LongTensor([src, dst])
    data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr, num_nodes=len(selected_nodes))

    return data, desc

# retrieve_pipeline.py

In [ ]:
import torch
import pandas as pd
from torch_geometric.data import Data
#from retrieval import retrieval_via_pcst

# Step 1 — Encode the query into an embedding
def encode_query(query_text, query_encoder):
    # query_encoder must be a model that outputs a vector from text
    with torch.no_grad():
        q_emb = query_encoder(query_text)  # returns a (feature_dim,) tensor
    return q_emb

# Step 2 — Prepare the graph (already assumed to be in PyG Data format)
def prepare_graph(graph_data):
    # graph_data must be a torch_geometric.data.Data object
    return graph_data

# Step 3 — Prepare textual node and edge tables
def prepare_textual_info(graph_data):
    # Dummy example: create node and edge DataFrames
    textual_nodes = pd.DataFrame({'node_feature': graph_data.x.tolist()})
    textual_edges = pd.DataFrame({
        'src': graph_data.edge_index[0].tolist(),
        'dst': graph_data.edge_index[1].tolist(),
        'edge_attr': graph_data.edge_attr.tolist()
    })
    return textual_nodes, textual_edges

# Step 4 — Retrieval via PCST
def retrieve_subgraph(query_text, query_encoder, graph_data):
    q_emb = encode_query(query_text, query_encoder)
    textual_nodes, textual_edges = prepare_textual_info(graph_data)
    subgraph, description = retrieval_via_pcst(
        graph=graph_data,
        q_emb=q_emb,
        textual_nodes=textual_nodes,
        textual_edges=textual_edges,
        topk=3,
        topk_e=3,
        cost_e=0.5
    )
    return subgraph, description

# Example usage:
# query_encoder = ... (your trained model or any text embedder)
# graph_data = ... (your loaded graph)
# subgraph, desc = retrieve_subgraph("example query", query_encoder, graph_data)


# compute graph embeddings & retrieve the most relevant graph:

In [34]:
import torch
from sentence_transformers import SentenceTransformer
from types import SimpleNamespace
#from src.model.graph_encoder import GraphEncoder
#from src.model.gnn import load_gnn_model

# 1. Load text encoder
text_encoder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")  # Or another model

# 2. Define GNN parameters
graph_encoder_args = SimpleNamespace(
    gnn_model_name="gat",
    gnn_in_dim=384,        # SBERT dim
    gnn_hidden_dim=128,
    gnn_num_layers=2,
    gnn_dropout=0.1,
    gnn_num_heads=4,       # Needed for GAT
)

# 3. Create the graph encoder
graph_encoder = GraphEncoder(graph_encoder_args)

graph_encoder.eval()  # Disable dropout during inference

# 4. Encode all graphs into fixed-size graph representations
'''graph_reprs = []
for graph in toy_graph_batch:
    with torch.no_grad():
        repr = graph_encoder.encode(graph)  # add batch dimension if needed
        graph_reprs.append(repr.squeeze(0))  # shape: [hidden_dim]

graph_reprs = torch.stack(graph_reprs)  # shape: [num_graphs, hidden_dim]'''
# ===

# 4.1. Transform all graphs to PyG format
pyg_graphs, _, _ = transform_graphs_to_pyg(toy_graphs,feature_dim=384)

# 4.2. Create one batch containing all graphs
batch = Batch.from_data_list(pyg_graphs)

# 4.3. Encode all graphs in one forward pass (most efficient)
with torch.no_grad():
    graph_reprs = graph_encoder.encode(batch)
# ===
'''
# Test each graph separately
for i, data in enumerate(pyg_graphs):
    try:
        single_batch = Batch.from_data_list([data])
        with torch.no_grad():
            graph_encoder.encode(single_batch)
        print(f"Graph {i} processed successfully")
    except Exception as e:
        print(f"Error in graph {i}: {str(e)}")'''

# 5. Encode the query
query = "What are common treatments for asthma?"
q_emb = torch.tensor(text_encoder.encode(query, convert_to_tensor=True))  # shape: [hidden_dim]
# Process query with projection
text_projection = nn.Linear(384, 4096)
with torch.no_grad():
    q_emb = text_encoder.encode(query, convert_to_tensor=True)
    q_emb = text_projection(q_emb)  # Now shape: [4096]

# 6. Compare and select the most similar graph
sims = torch.nn.functional.cosine_similarity(q_emb.unsqueeze(0), graph_reprs)  # shape: [num_graphs]
top_idx = torch.argmax(sims).item()

# 7. Get the most relevant graph
selected_graph = toy_graph_batch[top_idx]
#selected_node_texts = toy_node_texts[top_idx]
#selected_edge_texts = toy_edge_texts[top_idx]

# Get the most similar graph
print(f"Most relevant graph index: {top_idx}")
matched_graph = toy_graphs[top_idx]
# Print with formatting
print("=== Most Similar Graph ===")
print(f"Match Score: {sims[top_idx]:.3f}")
print("\nNodes:")
print("- " + "\n- ".join(matched_graph["nodes"]))

print("\nEdges:")
for src, rel, dst in matched_graph["edges"]:
    print(f"{src} -- {rel} --> {dst}")


Most relevant graph index: 3
=== Most Similar Graph ===
Match Score: -0.010

Nodes:
- Chronic obstructive pulmonary disease (COPD)
- Smoking
- Shortness of breath
- Lung damage

Edges:
Smoking -- leads to --> Chronic obstructive pulmonary disease (COPD)
Chronic obstructive pulmonary disease (COPD) -- causes --> Shortness of breath
Chronic obstructive pulmonary disease (COPD) -- results in --> Lung damage


<ipython-input-34-e41efb9612b0>:58: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  q_emb = torch.tensor(text_encoder.encode(query, convert_to_tensor=True))  # shape: [hidden_dim]
